In [1]:
# Import dependencies
import pandas as pd
import numpy as np
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
import requests
import zipfile
import glob

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240


[WDM] - Driver [C:\Users\Boss\.wdm\drivers\chromedriver\win32\86.0.4240.22\chromedriver.exe] found in cache


In [3]:
# URL of page to be scraped
url = 'https://s3.amazonaws.com/tripdata/index.html'

# Use splinter to navigate the site
browser.visit(url)

In [4]:
# Create BeautifulSoup object; parse with 'lxml'
html = browser.html
soup = bs(html, 'lxml')

In [5]:
# Get the links to download the data (excluding JC (short for Jersey City),
# and data for 2013 and 2014 which might be too far away)

links = [link["href"] for link in soup.find_all('a') if not any(
    s in link["href"] for s in ['index', 'JC', '2013', '2014'])]

In [6]:
# Download and unzip the zip files in the links
for link in links:
    print(f"Downloading {link}")
    r = requests.get(link)
    name = link.rsplit("/", 1)[-1]
    with open(f"data/{name}", 'wb') as f:
        f.write(r.content)

    with zipfile.ZipFile(f"data/{name}", 'r') as zip_ref:
        zip_ref.extractall("data/")

In [ ]:
col_names = pd.concat([pd.read_csv(f) for f in glob.glob(r'data/*.csv')], ignore_index = True).columns
col_names

In [ ]:
# Read in and concatenate all csv files
df = pd.concat([pd.read_csv(f) for f in glob.glob(r'data/*.csv')], ignore_index = True)
df.head()

In [ ]:
df.info()

In [ ]:
df.to_csv("data/conso_file.csv", index=False)